In [2]:
import tensorflow as tf
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
if __name__ == "__main__":
    sys.path.append("../../")
from notebooks_src.configs import configs
from notebooks_src.box_processing.tf_box_util import make_actual_gt_box_mask,ious_with_reference_boxes, convert_to_xyminmax, encode_to_scaled_offsets,\
convert_to_yxhw, make_box_coords_relative, preprocess_box_coordinates,zero_out_negatives
import numpy as np
from make_anchors_orig import make_anchors_dict, make_anchors

importing Jupyter notebook from make_anchors_orig.ipynb


In [5]:
def match_boxes(bboxes):
    """bboxes: numpy array B x max_boxes x 4 box coordinates for each batch, where B is number of Batches
                * optionally may have -1's where there is no box (unnormalized by im size)"""
    actual_gt_box_mask = make_actual_gt_box_mask(bboxes)
    anchors_dict = make_anchors_dict()
    mask_dict = {fmap_shape: dict(zip(["x_mask", "tp_mask", "num_matches","actual_gt_box_mask"],_match_boxes(bboxes, anchors,actual_gt_box_mask))) 
                                       for fmap_shape, anchors 
                                             in anchors_dict.iteritems()                
                   
                }
    return mask_dict

def _match_boxes(bboxes, anchors,actual_gt_box_mask, matching_threshold=0.5):
    """bboxes: numpy array B x max_boxes x 4 box coordinates for each batch, where B is number of Batches
                   * 0's where no box, normalized 
       anchors_for_layer: list of 4 arrays (y,x,h,w)
           * y and x is fm x fn x 1
           * h and w are (M,) where M is number of anchor box shapes"""
    aymin,  axmin, aymax, axmax = preprocess_anchors(anchors)
    ymin,xmin, ymax, xmax = preprocess_box_coordinates(bboxes)
    ious = ious_with_reference_boxes(reference_boxes=[aymin, aymax, axmin, axmax],axis=-1, bbox=[ymin,ymax,xmin,xmax])
    x_mask, tp_mask = make_x_mask(ious, actual_gt_box_mask, matching_threshold)
    num_matches = get_num_matches(x_mask)
    return x_mask, tp_mask, num_matches, actual_gt_box_mask

def get_num_matches(x_mask):
    float_x_mask = tf.cast(x_mask,dtype=tf.float32)        
    num_matches = tf.reduce_sum(float_x_mask)
    return num_matches
    
def preprocess_anchors(anchors):
    # anchor processing
    ay, ax, ah, aw = anchors
    aymin, aymax, axmin, axmax= convert_to_xyminmax(ay, ax, ah, aw)
    #pad with two dims at end of 1
    aymin, aymax, axmin, axmax = [np.expand_dims(np.expand_dims(tens,axis=-1), axis=-1)\
                                  for tens in [aymin, aymax, axmin, axmax]]
    return aymin, axmin, aymax ,axmax

def make_x_mask(ious, actual_gt_box_mask, matching_threshold):
    max_iou_for_each_box = tf.reduce_max(ious, axis=[0,1,2])

    best_box_mask = tf.greater_equal(x=ious, y=max_iou_for_each_box)

    thresh_mask = tf.greater_equal(x=ious, y=matching_threshold)

    tp_mask = tf.logical_or(thresh_mask, best_box_mask)

    x_mask = tf.logical_and(actual_gt_box_mask, tp_mask)

    x_mask = tf.transpose(x_mask, perm=[3,0,1,2,4])

    #X_mask is (batch_size, y, x, num_anchors, max_boxes(15))
    return x_mask, tp_mask

In [6]:
def save_make_mask_im(mask, name, num_splits, split_axis):
    mask_ims = tf.split(mask,num_or_size_splits=num_splits, axis=split_axis)
    for i,mask_im in enumerate(mask_ims):
        tf.summary.image(name=name + "_"+ str(i),tensor=mask_im)

In [13]:
if __name__ == "__main__":
    anchors = make_anchors_dict()
    a = preprocess_anchors(anchors[(96,144)])
#     import h5py
#     bbox = h5py.File(configs["tr_data_file"])["boxes"][7:12,:,:4]
#     x,am= match_boxes(bbox)
    
#     with tf.Session() as sess:
#         print sess.run(x[(96,144)]).shape